In [6]:
import pandas as pd
import tensorflow as tf
import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import os
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.densenet import preprocess_input, decode_predictions, DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.densenet import preprocess_input, decode_predictions, DenseNet121

In [7]:
csv_file_path = '/kaggle/input/butterfly-images40-species/butterflies and moths.csv'
df = pd.read_csv(csv_file_path)

print("Original DataFrame:")
print(df.head())

sorted_df = df.sort_values(by='filepaths')
sorted_df.reset_index(drop=True, inplace=True)

print("\nSorted DataFrame:")
print(sorted_df.head())

Original DataFrame:
   class id             filepaths  labels data set
0         0  train/ADONIS/001.jpg  ADONIS    train
1         0  train/ADONIS/002.jpg  ADONIS    train
2         0  train/ADONIS/003.jpg  ADONIS    train
3         0  train/ADONIS/004.jpg  ADONIS    train
4         0  train/ADONIS/005.jpg  ADONIS    train

Sorted DataFrame:
   class id          filepaths  labels data set
0         0  test/ADONIS/1.jpg  ADONIS     test
1         0  test/ADONIS/2.jpg  ADONIS     test
2         0  test/ADONIS/3.jpg  ADONIS     test
3         0  test/ADONIS/4.jpg  ADONIS     test
4         0  test/ADONIS/5.jpg  ADONIS     test


In [8]:
random_row = sorted_df.sample(n=1).iloc[0]
image_path = random_row['filepaths']
image_path = os.path.join('/kaggle/input/butterfly-images40-species', image_path)
print(f"Randomly selected image: {image_path}")

Randomly selected image: /kaggle/input/butterfly-images40-species/train/GREAT EGGFLY/060.jpg


In [76]:
"""
def load_images_from_directory(directory):
    image_data = []
    image_labels = []
    
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(('jpg', 'jpeg', 'png')):
                image_path = os.path.join(root, file)
                label = os.path.basename(root)
                
                try:
                    
                    image = load_img(image_path, target_size=(224, 224))
                    image = img_to_array(image)
                    image = preprocess_input(image)
                    
                    image_data.append(image)
                    image_labels.append(label)
                    
                except Exception as e:
                    print(f"Error processing image {image_path}: {e}")
       
    return np.array(image_data), np.array(image_labels)

train_directory = "kaggle/input/train"

x_train, y_train = load_images_from_directory(train_directory)

test_directory = "kaggle/input/test"
x_test, y_test = load_images_from_directory(test_directory)

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

y_train_categorical = to_categorical(y_train_encoded, num_classes=100)
y_test_categorical = to_categorical(y_test_encoded, num_classes=100)
""""

In [9]:
def load_images_from_directory(directory):
    image_data = []
    image_labels = []
    
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(('jpg', 'jpeg', 'png')):
                image_path = os.path.join(root, file)
                label = os.path.basename(root)
                try:
                   # Load and preprocess the image
                    image = load_img(image_path, target_size=(224, 224))
                    image = img_to_array(image)
                    image = preprocess_input(image)
                  
                   # Append to lists
                    image_data.append(image)
                    image_labels.append(label)
                except Exception as e:
                    print(f"Error processing image {image_path}: {e}")
    return np.array(image_data), np.array(image_labels)

In [10]:
# Load training data
train_directory = '/kaggle/input/butterfly-images40-species/train'
x_train, y_train = load_images_from_directory(train_directory)


# Load testing data
test_directory = '/kaggle/input/butterfly-images40-species/test'
x_test, y_test = load_images_from_directory(test_directory)


# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

y_train_categorical = to_categorical(y_train_encoded, num_classes=100)
y_test_categorical = to_categorical(y_test_encoded, num_classes=100)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Flatten


# Step 3: Load the DenseNet121 model
model = DenseNet121(weights='imagenet', include_top = False, input_shape = (224, 224, 3))

for layer in model.layers:
    layer.trainable = False

In [11]:
x = model.output
x = Flatten()(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation = 'relu')(x)
predictions = Dense(100, activation = 'sigmoid')(x)

model = Model(inputs = model.input, outputs = predictions)

model.compile(optimizer= 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

history = model.fit(x_train, y_train_categorical, epochs=10, batch_size=32)

Epoch 1/10
  1/394 ━━━━━━━━━━━━━━━━━━━━ 3:58:14 36s/step - accuracy: 0.0000e+00 - loss: 6.6648

I0000 00:00:1720019070.710685     270 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1720019070.782470     270 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


394/394 ━━━━━━━━━━━━━━━━━━━━ 91s 138ms/step - accuracy: 0.4227 - loss: 10.7189
Epoch 2/10


W0000 00:00:1720019125.060558     270 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


394/394 ━━━━━━━━━━━━━━━━━━━━ 35s 90ms/step - accuracy: 0.8644 - loss: 0.3128
Epoch 3/10
394/394 ━━━━━━━━━━━━━━━━━━━━ 41s 90ms/step - accuracy: 0.7895 - loss: 0.1589
Epoch 4/10
394/394 ━━━━━━━━━━━━━━━━━━━━ 36s 90ms/step - accuracy: 0.6811 - loss: 0.1766
Epoch 5/10
394/394 ━━━━━━━━━━━━━━━━━━━━ 41s 90ms/step - accuracy: 0.5645 - loss: 0.1503
Epoch 6/10
394/394 ━━━━━━━━━━━━━━━━━━━━ 36s 90ms/step - accuracy: 0.4491 - loss: 0.1982
Epoch 7/10
394/394 ━━━━━━━━━━━━━━━━━━━━ 35s 90ms/step - accuracy: 0.3233 - loss: 0.2984
Epoch 8/10
283/394 ━━━━━━━━━━━━━━━━━━━━ 9s 90ms/step - accuracy: 0.3499 - loss: 0.2692 

KeyboardInterrupt: 

In [89]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Flatten

rand_row = sorted_df.sample(n=1).iloc[0]
img_path = rand_row['filepaths']
img_path = os.path.join('kaggle/input', img_path)

model = DenseNet121(weights = "imagenet", include_top=False, input_shape=(224,224,3))

for layer in model.layers:
    layer.trainable=False
    
x = model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(100, activation="sigmoid")(x)

model = Model(inputs=model.input, outputs = predictions)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train_categorical, epochs=10, batch_size=32)

Epoch 1/10


ValueError: Exception encountered when calling Functional.call().

[1mInvalid input shape for input Tensor("data:0", shape=(32,), dtype=float32). Expected shape (None, 224, 224, 3), but input has incompatible shape (32,)[0m

Arguments received by Functional.call():
  • inputs=tf.Tensor(shape=(32,), dtype=float32)
  • training=True
  • mask=None

In [18]:
base_model = Sequential([
    keras.Input(shape=(224,224,3)),
    Conv2D(20, (5,5), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(40, (5,5), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dropout(0.5),
    Dense(640,activation='relu'),
    Dropout(0.5),
    Dense(1000,activation='relu'),
    Dropout(0.5),
    Dense(10,activation='softmax')
])

base_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
base_model.fit(x_train,y_train,epochs=40,batch_size=10)

score = base_model.evaluate(x_test,y_test,batch_size=10)

NameError: name 'keras' is not defined